In [1]:
# finetuning llama2

# (Q)分子構造+(R)理由+(A)物性データセットのLLMによる学習と予測
- Q&A: 融点データセットを使用
- R: GPT4を使い､Q&Aをもとに自動生成
- 強化学習をしてみる

In [1]:
#!pip install transformers==4.35.0
#!pip install peft==0.5.0
#!pip install bitsandbytes==0.41.1
#!pip install accelerate==0.23.0
#!pip install flash-attn==2.3.1.post1
#!pip install datasets==2.14.5

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer,pipeline
from datasets import Dataset
import copy
from tqdm import tqdm
#問題設定: はじめのN件をテストデータにする
n_test=50

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#ハイパラ関連
#モデル名
model_size=7
model_size=13
#model_size=70
model_name=f"meta-llama/Llama-2-{model_size}b-chat-hf"

#LoRA関連
r=32
lora_alpha=r
bit=16
#bit=8
#bit=4

#LoRAのadapter
target_modules= [
    #"embed_tokens",
    "lm_head",
    #"q_proj",
    #"k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    #"down_proj",
]

#学習関連
#gradient_checkpointing =True  #vramの節約をしたい場合
gradient_checkpointing =False
per_device_train_batch_size=1
epochs=1
lr=10**-5
do_train=True
#do_train=False

In [3]:

device_map="auto"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

def init_model(model_name, r, lora_alpha, target_modules, bit=4):
    if bit == 4:
        print("Using 4-bit mode")
        model = AutoModelForCausalLM.from_pretrained(model_name,
                                                     quantization_config=bnb_config,
                                                     device_map=device_map,
                                                     use_flash_attention_2=True,
                                                     )
    elif bit == 8:
        print("Using 8-bit mode")
        model = AutoModelForCausalLM.from_pretrained(model_name,
                                                     load_in_8bit=True,
                                                     device_map=device_map,
                                                     use_flash_attention_2=True,
                                                     )
    elif bit == 16:
        print("Using fp16 mode")
        model = AutoModelForCausalLM.from_pretrained(model_name,
                                                     device_map=device_map,
                                                     torch_dtype=torch.float16,
                                                     use_flash_attention_2=True,
                                                     )
    else:
        raise ValueError("bit must be 4 or 16")

    if len(target_modules)==0:
        return model
    peft_config = LoraConfig(
        task_type="CAUSAL_LM", inference_mode=False, r=r, lora_alpha=lora_alpha,
        lora_dropout=0.1,
        target_modules=target_modules,
    )
    model = get_peft_model(model, peft_config)
    return model


In [4]:

#モデル初期化
model=init_model(model_name, r, lora_alpha, target_modules, bit=bit)

Using fp16 mode


In [4]:


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

#pipe = pipeline("text-generation", model=model,
#                tokenizer=tokenizer, max_new_tokens=1000)

# データセットの生成

In [5]:
#データセットの読み込み
import pandas as pd
import random
df=pd.read_csv("dataset/231225AutoReasoning/240104best_reason_record.csv")
dataset=df.to_dict(orient="records")
random.seed(0)
random.shuffle(dataset)

print(df.shape)
df[:2]

(2406, 9)


,name,smiles,csid,link,source,Reason,mpC,Prediction(integer),Abs error
0,"2,3-dibromo-2-methyl-butane",BrC(C)(C)C(Br)C,71392,http://www.srcinc.com/what-we-do/product.aspx?...,PHYSPROP,"Starting with butane, which melts around -138....",7.0,6.6,0.4
1,5-chlorobenzofuroxan,c1cc2c(cc1Cl)no[n+]2[O-],123661,http://www.alfa.com/en/GP100W.pgm?DSSTK=A14261,Alfa Aesar,"The basic structure for comparison is benzene,...",47.0,45.0,2.0


In [6]:
import random
system_prompt="You are a professional chemist. Predict the melting point of the following compound."


def gen_compound_text(chemical_record,
    reason="",prediction=""):
    name=chemical_record["name"]
    smiles=chemical_record["smiles"]
    prompt=f"""
#Problem
##Name: {name}
##SMILES: {smiles}"""
    if reason !="" and prediction!="":
        prompt+=f"""
##Reason: {reason}
##Prediction: {prediction}
"""
    else:
        #test mode
        prompt+="""
##Reason: 
"""
    return prompt



def generate_question_prompt(dataset,test_id,n_prompt_examples=5):
    train_ids=[i for i in range(len(dataset))]
    train_ids.remove(test_id)
    prompt=""

    #train prompt
    for _ in range(n_prompt_examples):
        id=random.choice(train_ids)
        prompt+=gen_compound_text(dataset[id],
                                reason=dataset[id]["Reason"],
                                prediction=dataset[id]["Prediction(integer)"])
        prompt+="\n"

    #test prompt
    prompt+=gen_compound_text(dataset[test_id])

    return prompt


def prepare_dataset(context_list, tokenizer):
    data_list = [{"text": i} for i in context_list]
    random.shuffle(data_list)

    # tokenize
    dataset = Dataset.from_dict(
        {"text": [item["text"] for item in data_list[:]]})
    dataset = dataset.map(lambda samples: tokenizer(
        samples['text']), batched=True)

    return dataset


In [7]:


train_text_list=[]
for id in range(len(dataset)):
    prompt=gen_compound_text(dataset[id],
                                reason=dataset[id]["Reason"],
                                prediction=dataset[id]["Prediction(integer)"])
    train_text_list.append(prompt)
tokenized_dataset = prepare_dataset(train_text_list[n_test:], tokenizer)

Map: 100%|██████████| 2356/2356 [00:00<00:00, 8903.46 examples/s]


In [8]:
#check prompt

print("train")
print(prompt)
print("test")
t_prompt=gen_compound_text(dataset[0])
print(t_prompt)

train

#Problem
##Name: 2-Ethoxy-2-methylpropane
##SMILES: CC(C)(C)OCC
##Reason: The target compound, 2-Ethoxy-2-methylpropane, can be seen as derived from propane, which has a melting point of -187.7°C. Adding a methyl group generally increases the melting point due to increased molecular weight and van der Waals forces. For propane, adding a methyl group to become isobutane elevates the melting point to -159.6°C, indicating the effect of a methyl group is an increase of roughly +28°C. The presence of an ethoxy group (-OCC2H5) introduces an oxygen atom which can lead to dipole-dipole interactions and potential hydrogen bonding with trace moisture, increasing the melting point as well. The ethoxy group's effect on the melting point would be less straightforward to quantify without empirical data, but we can predict that it will raise the melting point to some extent. Considering the combined effect of an additional methyl group and an ethoxy group on the basic structure of propane, we 

# モデルの訓練

In [11]:
import transformers
from datetime import datetime


#train
train_args = transformers.TrainingArguments(
        per_device_train_batch_size=per_device_train_batch_size,
        #gradient_accumulation_steps=1,
        warmup_steps=0,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=True,
        logging_steps=100,
        save_total_limit=1,
        output_dir='outputs/'+datetime.now().strftime('%Y%m%d%H%M%S'),
        gradient_checkpointing=gradient_checkpointing,
    )

# trainer
#callbacks = [EarlyStoppingCallback()]
callbacks = []

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=train_args,
    callbacks=callbacks,
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, mlm=False)
)

if do_train:
    training_result = trainer.train()
    training_result.training_loss

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,1.690200
200,1.272200
300,1.165200
400,1.097800
500,1.095800
600,1.061700
700,1.034000
800,1.047700
900,1.037100
1000,1.022400


In [9]:
#モデルの保存: adapterのみ保存するとき｡
from datetime import datetime
current_datetime = datetime.now()
#model.save_pretrained(f"./outputs/{current_datetime}")
#model.save_pretrained(f"./outputs/7b_ft")

In [10]:
#モデルの読み込み: 強化学習しない場合
from peft import AutoPeftModelForCausalLM
model_path="./outputs/7b_ft"

"""
model = AutoPeftModelForCausalLM.from_pretrained(model_path,
                                                 device_map=device_map,
                                                     torch_dtype=torch.float16,
                                                     use_flash_attention_2=True,
                                                 )

"""

'\nmodel = AutoPeftModelForCausalLM.from_pretrained(model_path,\n                                                 device_map=device_map,\n                                                     torch_dtype=torch.float16,\n                                                     use_flash_attention_2=True,\n                                                 )\n\n'

In [11]:
#モデルのマージと保存: 強化学習するとき
#merged_model = model.merge_and_unload()
#merged_model.save_pretrained(f"./outputs/13b_ft_merge")

# 強化学習による追加訓練

In [12]:
#予測周りのutility funcs
import re
import torch
import gc
from IPython.display import clear_output
from trl import AutoModelForCausalLMWithValueHead
def gen_text_stop_word(prompt,model,tokenizer,
                       device="cuda:0",
                       stop_words=["#Problem","#Reason","# Problem"],
                       double_stop_words=["#Prediction"],
                       stream=False,
                       #stream=True,
                       max_tokens=300,
                       ):
    gc.collect()
    torch.cuda.empty_cache()



    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    # 生成されたテキストを格納する変数
    generated_text = ""

    # トークンを一つずつ生成
    for i in range(max_tokens):
        # 次のトークンを予測
        outputs = model(input_ids)
        if type(model) is AutoModelForCausalLMWithValueHead:
            #AutoModelForCausalLMWithValueHeadの場合
            logits = outputs[0]
            next_token_logits = logits[:, -1, :]
        else:
            next_token_logits = outputs.logits[:, -1, :]

        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)

        # 生成されたトークンを現在の入力に追加
        input_ids = torch.cat([input_ids, next_token], dim=-1)

        # 生成されたテキストを更新
        generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)[len(prompt):]

        if stream:
            if i%30==0:
                clear_output()
            print(generated_text)

        # ストップワードのチェック
        if any(stop_word in generated_text for stop_word in stop_words):
            break

        # 2回以上出現したらstopするwordのcheck 
        stop_flag=False
        for check_word in double_stop_words:
            count=generated_text.count(check_word)
            if count>=2:
                stop_flag=True
                break
        if stop_flag:
            break

    return generated_text

def ask_value(prompt,model,tokenizer):
    res=gen_text_stop_word(prompt,model,tokenizer)
    #res=pipe(prompt)[0]["generated_text"]
    print("----\n\n")
    print(res.strip())

    regex_list=[
        r"Prediction:\s*(\d+\.?\d*)",
        r"Prediction:\s*(-?\d+\.?\d*)",
    ]

    value=None
    for reg in regex_list:
        match = re.search(reg, res)
        if match:
            value = match.group(1)
            break


    return res,value



In [13]:
model_path="./outputs/7b_ft_merge"
#model_path="./outputs/13b_ft_merge"

#テキストでファインチューニングしたモデルを強化学習用に読み込み
lora_config = LoraConfig(
    target_modules=target_modules,
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = AutoModelForCausalLMWithValueHead.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map=device_map,
    peft_config=lora_config,
)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.02it/s]


In [14]:
from trl import PPOTrainer, PPOConfig
#trainerの定義
ppo_trainer = PPOTrainer(
    config=PPOConfig(batch_size=1),
    model=model, 
    #ref_model=ref_model, 
    tokenizer=tokenizer,
    )

/home/setup/miniconda3/envs/chemllm/lib/python3.10/site-packages/trl/trainer/ppo_trainer.py:257: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [15]:
#報酬クラスの定義
class RewardModel:
    def __init__(self,model,dataset,tokenizer,
                 range_value=1,
                 reward_slope=100,
                 n_prompt_examples=3,
                 device="cuda:0",
                 ):
        self.model=model
        self.dataset=dataset
        self.tokenizer=tokenizer    
        self.n_prompt_examples=n_prompt_examples
        self.range_value = range_value
        self.reward_slope = reward_slope
        self.device=device

    def calculate_reward(self,abs_error, ):
        """
        Calculate the reward based on the absolute error.

        :param abs_error: The absolute error value.
        :param range_value: The range value for maximum and minimum rewards.
        :return: The calculated reward.
        """

        reward = self.range_value - (abs_error) * (self.range_value / self.reward_slope)
        return max(reward, -self.range_value)


    def __call__(self,train_id):
        prompt=generate_question_prompt(self.dataset,train_id,
                                        n_prompt_examples=self.n_prompt_examples)
        reason,value=ask_value(prompt,self.model,self.tokenizer,
                                         )

        actual=self.dataset[train_id]["mpC"]
        if value is None:
            reward=-self.range_value
        else:
            value=float(value)

            abs_error=abs(actual-value)
            # Example usage of the function
            reward=self.calculate_reward(abs_error)  # Example case where abs_error is 12 and range_value is 100

        print(f"actual: {actual}, predicted: {value}, reward: {reward}")

        input_ids = tokenizer.encode(prompt, return_tensors='pt').to(self.device)
        return reward,reason,input_ids


In [16]:

reward_model = RewardModel(model,dataset,tokenizer)
reward_model.calculate_reward(30)  #報酬の動作確認

0.7

In [17]:
import warnings
warnings.filterwarnings('ignore')
model.config.use_cache = False 
lr_epochs=250
#lr_epochs=3

for i in tqdm(range(lr_epochs)):

    #ランダムに問題を設定して値を予測させる
    train_id=random.randint(n_test,len(dataset))


    #同じ問題に対して､良い結果が来るまで何回か問題を解かせる
    for i in range(3):
        #報酬､応答､入力の取得
        reward,response,input_id=reward_model(train_id)
        rewards=[torch.tensor(float(reward))]
        query_tensors = [torch.tensor(input_id).reshape(-1)]
        response_tensors=[torch.tensor(tokenizer.encode(response)).reshape(-1)]

        #モデル更新
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

        #結果がよかったら次の問題に移る
        if reward>0.5:
            break

  0%|          | 0/250 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


----


The target compound, (2Z)-6-methylhept-5-en-2-one semicarbazone, has several functional groups that affect its melting point. 1) The basic unit, hept-5-en-2-one, has a melting point of -42°C. 2) The methyl group (CH3) adds steric bulk and increases the molecular weight, estimated to be +10. 3) The carbonyl group (C=O) contributes to the rigidity of the molecule and can form hydrogen bonds, estimated to be +20. 4) The semicarbazone moiety (C=NN=C(C)CCC=C(C)C) introduces additional polarity and potential for hydrogen bonding, estimated to be +30. Base molecule hept-5-en-2-one has a melting point of -42°C for reference. Adding up these contributions, we get a predicted melting point shift of +10 (methyl group) + 20 (carbonyl group) + 30 (semicarbazone) compared to the base hept-5-en-2-one's melting point.
##Prediction: 10.0


#Problem
actual: 136.0, predicted: 10.0, reward: -0.26
----


The compound (2Z)-6-methylhept-5-en-2-one semicarbazone has several functional groups that influ

  0%|          | 1/250 [05:34<23:06:39, 334.14s/it]

----


The compound in question, 2-Deoxy-2-[(methoxycarbonyl)amino]-beta-D-glucopyranose, has several functional groups that affect its melting point. Starting with glucose, which has a melting point of 140.0 °C, we consider the effects of the functional groups. The deoxyribose structure is less rigid than ribose, which could lower the melting point by approximately -20°C. The methoxycarbonyl group introduces a polar carbonyl group, which can participate in hydrogen bonding, raising the melting point by approximately +20°C. The amino group can form hydrogen bonds and participate in intermolecular interactions, estimated to add +30°C. The presence of the glucopyranose ring and the hydroxyl group at the 2-position can also contribute to hydrogen bonding, estimated to add +20°C. However, the steric hindrance and the potential for intramolecular hydrogen bonding might restrict the ability of the functional groups to engage in extensive intermolecular hydrogen bonding, which could lead to a

...

 99%|█████████▉| 248/250 [22:18:10<11:44, 352.36s/it]

----


mű alt privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile pr

100%|█████████▉| 249/250 [22:24:32<06:01, 361.26s/it]

----


Kontrola privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile 

100%|██████████| 250/250 [22:29:46<00:00, 323.95s/it]


In [18]:
from datetime import datetime
current_datetime = datetime.now()
#model.save_pretrained(f"outputs/LR7")
#model.save_pretrained(f"./outputs/{current_datetime}")

# モデルによる物性値の予測

In [19]:

model.eval()

AutoModelForCausalLMWithValueHead(
  (pretrained_model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(32000, 4096)
          (layers): ModuleList(
            (0-31): 32 x LlamaDecoderLayer(
              (self_attn): LlamaAttention(
                (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (k_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (v_proj): Linear8bitLt(
                  in_features=4096, out_features=4096, bias=False
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=32, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=32, out_features=

In [20]:
n_prompt_examples=3 #何件の例題をprompt tuningで出すか
n_max_trials=3  # 値を返さなかったときの再試行の最大数
random.seed(0)
prediction_results={}

#予測時のハイパラ
n_prompt_examples=3 #何件の例題をprompt tuningで出すか
n_max_trials=3  # 値を返さなかったときの再試行の最大数

res_list=[]
for test_id in tqdm(range(n_test)):
    print(f"promlem {test_id+1} / {n_test}")
    for _ in range(n_max_trials):
        try:
            prompt=generate_question_prompt(dataset,test_id,n_prompt_examples=n_prompt_examples)
            reason,value=ask_value(prompt,model,tokenizer)
        except Exception as e:
            print(e)
            continue


        if value is not None:
            record=copy.deepcopy(dataset[test_id])
            record["Test (Predicted reason)"]=reason
            record["Test (Predicted value)"]=value
            res_list.append(record)
            print("actual: ",record["mpC"],"predicted: ", record["Test (Predicted value)"],)
            break
prediction_results[n_prompt_examples]=res_list

  0%|          | 0/50 [00:00<?, ?it/s]

promlem 1 / 50
----


!! Squad privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile p

  2%|▏         | 1/50 [05:37<4:35:42, 337.60s/it]

----


Kontrola privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile 

  4%|▍         | 2/50 [10:40<4:13:52, 317.34s/it]

...

----


Kontrola privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile 

 98%|█████████▊| 49/50 [4:44:55<05:46, 346.83s/it]

----


Kontrola privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile 

100%|██████████| 50/50 [4:50:49<00:00, 348.99s/it]

----


!!fiddle media privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile privile pr

In [21]:
#plot
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from datetime import datetime
import json
current_datetime = datetime.now()
vmin=-200
vmax=300

#plot prediction results
for n_prompt_examples,records in prediction_results.items():
    sel_df=pd.DataFrame(records)
    #floatに可能なものは変換
    sel_df["Test (Predicted value)"] = pd.to_numeric(sel_df["Test (Predicted value)"], errors='coerce')
    sel_df=sel_df[sel_df["Test (Predicted value)"].notnull()]
    if len(sel_df)==0:
        continue
    mse=mean_squared_error(sel_df["mpC"],sel_df["Test (Predicted value)"])

    plt.figure()
    sns.scatterplot(data=sel_df,x="mpC",y="Test (Predicted value)")
    plt.title(f"n_prompt_examples={n_prompt_examples} MSE={mse:.0f}")

    #x,yの範囲を揃える
    plt.xlim(vmin,vmax)
    plt.ylim(vmin,vmax)
    #対角線を描く
    plt.plot([vmin,vmax],[vmin,vmax],color="gray")
    formatted_filename = f"results/model={model_size}_{current_datetime.strftime('%Y%m%d_%H%M%S')}_train={do_train}.png"
    plt.savefig(formatted_filename)
    #break

save_json_filename=formatted_filename.replace(".png",".json")
with open(save_json_filename,"w") as f:
    json.dump(prediction_results,fp=f,
              indent=4)

KeyError: 'Test (Predicted value)'

In [ ]:
#スコア
print(mean_squared_error(sel_df["mpC"],sel_df["Test (Predicted value)"]))
print(mean_absolute_error(sel_df["mpC"],sel_df["Test (Predicted value)"]))
print(r2_score(sel_df["mpC"],sel_df["Test (Predicted value)"]))


8153.85
81.3
-6.077011873350925


In [ ]:
#回答可能な問題の割合
sel_df.shape[0]/n_test

0.1